# E-commerce Customer Analysis Project

# The Look Ecommerce Data Analysis

# Analyst: Ariku Charles

# Client: The Look Online Clothing Store

# Date: 11 2025


# E-commerce Customer Analysis Project

# Objective

To analyze The Look's e-commerce data to identify high-value customer segments, optimize marketing channel allocation, improve product profitability, and streamline operational efficiency, ultimately increasing customer lifetime value and overall business profitability.

# Problem Statement

The Look faces multiple interconnected challenges affecting profitability and growth:

Ineffective marketing spend with unclear ROI by channel

Suboptimal product performance and high return rates

Operational inefficiencies in fulfillment processes

Inability to identify and retain high-value customers

Disconnected insights across business functions

# Key Questions to Answer

Which marketing channels deliver customers with the highest lifetime value?

What product characteristics correlate with high profitability and low returns?

Where are the bottlenecks in the order fulfillment process?

Which customer segments demonstrate the highest completion rates and loyalty?

How can operational processes be optimized to reduce cancellations and returns?

# Success Metrics

Order completion rates.

Return rates.

Improve customer lifetime value

Optimize marketing channel ROI

Enhance operational efficiency

# Deliverables

Customer segmentation analysis with RFM scoring

Marketing channel performance assessment

Product profitability and return analysis

Operational bottleneck identification

Actionable strategic recommendations

# Stakeholders
Executive leadership, marketing team, operations managers, product merchandising, customer experience, and finance department.

# Dataset
The dataset used is The Look E-commerce Dataset, a synthetic dataset from Looker simulating a fictional clothing retailer (7 interconnected CSV files: users, orders, order items, products, events, inventory items, and distribution centers). It's available as a public dataset on Google BigQuery and can be downloaded as CSV files from the BigQuery console or Kaggle. For this analysis, the CSV files were stored locally at E:\TheLookEcommerce.



# Data Sources

## Primary Dataset
**Name**: The Look E-commerce Dataset  
**Source**: Looker/Google BigQuery Public Datasets  
**Format**: 7 interconnected CSV files  
**Purpose**: Synthetic e-commerce data for analytics training

## Dataset Structure
| File | Rows | Description |
|------|------|-------------|
| `users.csv` | 100,000 | Customer profiles and demographics |
| `orders.csv` | 125,226 | Customer purchase records |
| `products.csv` | 29,120 | Product catalog and pricing |
| `order_items.csv` | 181,759 | Individual order line items |
| `events.csv` | 1,048,575 | Website user interactions |
| `inventory_items.csv` | 490,705 | Stock management |
| `distribution_centers.csv` | 10 | Warehouse locations |

## Data Quality (ROCCC)
- **Reliable**: Structured synthetic data from Google
- **Comprehensive**: Complete e-commerce ecosystem
- **Current**: Multi-year data with recent timestamps

# Data Cleaning & Manipulation
## Importing packages/Libraries
These Packages/libraries provide the necessary tools for comprehensive file handling and data analysis. Pandas enables efficient data manipulation, while scipy allows for statistical hypothesis testing. Visualization libraries help in exploratory data analysis and result presentation.

In [1]:
# UTILITIES
import os
from datetime import datetime
from pathlib import Path

# DATA MANIPULATION & ANALYSIS
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# VISUALIZATION
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# MACHINE LEARNING
import sklearn
# STATISTICS
from scipy import stats
import statsmodels.api as sm


# Data Loading with OOP Structure

Created a custom Python class TheLookEcommerce that loads all 7 CSV files as class attributes using pandas' read_csv() function.
Using Object-Oriented Programming (OOP) provides organized data management, keeps related datasets together, and allows for method encapsulation. This structure makes the data easily accessible and maintainable throughout the analysis.
The Main class With initialising function for the Attributes

In [11]:
# Redefine the entire class with the new method
class TheLookEcommerce:
    
    def __init__(self, data_path):
        # Load all 7 files directly in constructor
        self.distribution_centers = pd.read_csv(f"{data_path}/distribution_centers.csv")
        self.events = pd.read_csv(f"{data_path}/events.csv")
        self.inventory_items = pd.read_csv(f"{data_path}/inventory_items.csv")
        self.order_items = pd.read_csv(f"{data_path}/order_items.csv")
        self.orders = pd.read_csv(f"{data_path}/orders.csv")
        self.products = pd.read_csv(f"{data_path}/products.csv")
        self.users = pd.read_csv(f"{data_path}/users.csv")
    
    def check_files_loaded(self):
        """Check if all 7 files have loaded successfully"""
        
        print("Checking loaded files...")
        print("-" * 40)
        
        files = [
            ("distribution_centers", self.distribution_centers),
            ("events", self.events),
            ("inventory_items", self.inventory_items),
            ("order_items", self.order_items),
            ("orders", self.orders),
            ("products", self.products),
            ("users", self.users)
        ]
        
        for name, data in files:
            if data is not None:
                print(f"✅ {name}: {data.shape[0]} rows, {data.shape[1]} columns")
            else:
                print(f"❌ {name}: NOT LOADED")
        
        print("-" * 40)

In [12]:
thelook = TheLookEcommerce("E:/TheLookEcommerce")
thelook.check_files_loaded()

Checking loaded files...
----------------------------------------
✅ distribution_centers: 10 rows, 4 columns
✅ events: 1048575 rows, 13 columns
✅ inventory_items: 490705 rows, 12 columns
✅ order_items: 181759 rows, 11 columns
✅ orders: 125226 rows, 9 columns
✅ products: 29120 rows, 9 columns
✅ users: 100000 rows, 15 columns
----------------------------------------


### Initial Data Verification
Verified that all 7 files loaded successfully with their expected row and column counts, confirming data integrity from source to analysis environment.
Initial verification ensures that no data was corrupted during transfer or loading. Confirming expected dimensions provides confidence that the complete dataset is available for analysis.

# Data Structure Examination
Examined the structure of key tables including column names, data types, and basic statistics for numerical columns.
Understanding data structure is essential for selecting appropriate analysis methods. Different data types require different handling - numerical data allows statistical tests, while categorical data requires different analytical approaches.
We need to understand the structure of our data before cleaning it. 
### Starting with User Data Analysis
we will begin with the users.csv.Users are the core of e-commerce analysis. All business questions about sales, marketing, and growth ultimately connect back to customers. By examining users data first, we build a clean customer foundation that supports accurate analysis of purchases, browsing behavior, and preferences. This initial inspection checks data structure, types, missing values, and quality issues to ensure our customer insights are reliable before connecting to orders, events, and products.

In [37]:
     def inspect_users():
        """Check users data: structure, types, missing values, duplicates"""
        
        print("🔍 Users Data Inspection")
        print("-" * 40)
        
        data = thelook.users
        
        # 1. Structure
        print(f"Rows: {data.shape[0]:,}")
        print(f"Columns: {data.shape[1]}")
        
        # 2. Data types
        print("\n📋 Data Types:")
        for col, dtype in data.dtypes.items():
            print(f"  {col:20} - {dtype}")
        
        # 3. Missing values
        print("\n⚠️  Missing Values:")
        missing = data.isnull().sum()
        
        if missing.sum() == 0:
            print("  No missing values")
        else:
            for col in data.columns:
                if data[col].isnull().sum() > 0:
                    count = data[col].isnull().sum()
                    percent = (count / len(data)) * 100
                    print(f"  {col:20} - {count:,} ({percent:.1f}%)")
        
        # 4. Duplicate check
        print("\n🔍 Duplicate Check:")
        duplicate_ids = data['id'].duplicated().sum()
        print(f"  Duplicate customer IDs: {duplicate_ids}")
        
        print("-" * 40)
        return data
    
    # Run inspection
users_data = inspect_users()

🔍 Users Data Inspection
----------------------------------------
Rows: 100,000
Columns: 15

📋 Data Types:
  id                   - int64
  first_name           - object
  last_name            - object
  email                - object
  age                  - int64
  gender               - object
  state                - object
  street_address       - object
  postal_code          - object
  city                 - object
  country              - object
  latitude             - float64
  longitude            - float64
  traffic_source       - object
  created_at           - object

⚠️  Missing Values:
  city                 - 958 (1.0%)

🔍 Duplicate Check:
  Duplicate customer IDs: 0
----------------------------------------


# Traffic Source Analysis
Filtered and summarized users by their acquisition source using the traffic_source column. Aggregated customer counts and calculated percentage distribution across all five marketing channels.
Understanding customer acquisition patterns helps optimize marketing spend. By quantifying where customers come from, we can identify which channels drive the most volume and which may require strategic evaluation for improvement or reallocation of resources.

In [38]:
    def analyze_traffic_source():
        """Analyze where customers come from (traffic sources)"""
        
        print("📈 Traffic Source Analysis")
        print("-" * 40)
        
        data = thelook.users
        
        if 'traffic_source' not in data.columns:
            print("❌ No traffic_source column found")
            return
        
        # 1. Basic counts
        traffic_counts = data['traffic_source'].value_counts()
        
        print(f"Total customers: {len(data):,}")
        print(f"Unique traffic sources: {data['traffic_source'].nunique()}")
        
        print("\n📊 Customer Count by Source:")
        for source, count in traffic_counts.items():
            percent = (count / len(data)) * 100
            print(f"  {source:15} - {count:6,} customers ({percent:5.1f}%)")
        
        print("-" * 40)
        return traffic_counts

# Run analysis
traffic_summary = analyze_traffic_source()

📈 Traffic Source Analysis
----------------------------------------
Total customers: 100,000
Unique traffic sources: 5

📊 Customer Count by Source:
  Search          - 70,075 customers ( 70.1%)
  Organic         - 15,110 customers ( 15.1%)
  Facebook        -  5,816 customers (  5.8%)
  Email           -  4,947 customers (  4.9%)
  Display         -  4,052 customers (  4.1%)
----------------------------------------


The analysis shows the distribution of customers across five traffic sources, revealing that Search marketing dominates with 70.1% of customers, followed by Organic traffic at 15.1%. Facebook, Email, and Display channels collectively account for only 14.8% of the customer base.

This customer count distribution reveals little about customer quality, purchase behavior, or business value. High customer volume from Search doesn't indicate whether these customers complete purchases, have high lifetime value, or represent profitable acquisitions. We only know where customers come from, not what they do afterwords.

To address this limitation, we must aggregate traffic sources by order status to analyze how customers from different channels actually behave. This involves connecting user traffic source data with their order outcomes to determine completion rates, return rates, and cancellation patterns by acquisition channel. This deeper analysis will reveal whether high-volume channels also deliver high-quality customers and successful purchase outcomes.


In [39]:
    def analyze_order_status_by_traffic():
        """Analyze order item status by traffic source"""
        
        print("🔄 Order Status Analysis by Traffic Source")
        print("="*70)
        
        # Check we have all needed data
        if thelook.users is None or thelook.orders is None or thelook.order_items is None:
            print("❌ Missing required data (users, orders, or order_items)")
            return
        
        users = thelook.users
        orders = thelook.orders
        order_items = thelook.order_items
        
        # Step 1: Connect users to their orders
        print("Step 1: Connecting users to their orders...")
        
        # Merge orders with users to get traffic source
        orders_with_source = pd.merge(
            orders[['order_id', 'user_id']],
            users[['id', 'traffic_source']],
            left_on='user_id',
            right_on='id',
            how='left'
        )
        
        # Step 2: Connect to order items to get status
        print("Step 2: Connecting to order items...")
        
        # Merge with order items
        items_with_source = pd.merge(
            order_items[['order_id', 'status']],
            orders_with_source[['order_id', 'traffic_source']],
            on='order_id',
            how='left'
        )
        
        # Step 3: Analyze by traffic source
        print("Step 3: Analyzing by traffic source...\n")
        
        # Get unique traffic sources
        traffic_sources = items_with_source['traffic_source'].dropna().unique()
        
        results = []
        
        for source in traffic_sources:
            # Filter items for this traffic source
            source_items = items_with_source[items_with_source['traffic_source'] == source]
            total_items = len(source_items)
            
            if total_items == 0:
                continue
            
            # Count each status
            status_counts = source_items['status'].value_counts()
            
            # Get common statuses
            completed = status_counts.get('Complete', 0)
            returned = status_counts.get('Returned', 0) 
            
            # Calculate completion rate
            completion_rate = (completed / total_items) * 100 if total_items > 0 else 0
            
            results.append({
                'Traffic Source': source,
                'Total Items': total_items,
                'Completed': completed,
                'Returned': returned,
                'Cancelled': status_counts.get('Cancelled', 0),
                'Shipped': status_counts.get('Shipped', 0),
                'Processing': status_counts.get('Processing', 0),
                'Completion Rate': f"{completion_rate:.1f}%"
            })
        
        # Create results DataFrame
        results_df = pd.DataFrame(results)
        
        # Sort by completion rate (highest first)
        results_df['Sort_Rate'] = results_df['Completion Rate'].str.replace('%', '').astype(float)
        results_df = results_df.sort_values('Sort_Rate', ascending=False)
        results_df = results_df.drop('Sort_Rate', axis=1)
        
        # Display results
        print("📊 ORDER STATUS BY TRAFFIC SOURCE")
        print("-"*70)
        
        # Format the display
        pd.set_option('display.width', 1000)
        pd.set_option('display.max_columns', None)
        
        print(results_df.to_string(index=False))
        
        print("\n" + "="*70)
        print("🎯 KEY FINDINGS")
        print("="*70)
        
        # Find best and worst performers
        if not results_df.empty:
            best_source = results_df.iloc[0]
            worst_source = results_df.iloc[-1]
            
            print(f"🏆 BEST: {best_source['Traffic Source']} - {best_source['Completion Rate']} completion rate")
            print(f"   {best_source['Completed']:,} completed out of {best_source['Total Items']:,} items")
            
            print(f"\n⚠️  WORST: {worst_source['Traffic Source']} - {worst_source['Completion Rate']} completion rate")
            print(f"   {worst_source['Completed']:,} completed out of {worst_source['Total Items']:,} items")
            
            # Calculate average return rate
            print(f"\n📈 AVERAGE RETURN RATE BY SOURCE:")
            for _, row in results_df.iterrows():
                return_rate = (row['Returned'] / row['Total Items']) * 100
                print(f"   {row['Traffic Source']:10} - {return_rate:5.1f}% returned")
        
        print("="*70)
        
        return results_df
    
    # Run the analysis
    print("Starting analysis of order status by traffic source...")
    status_by_traffic = analyze_order_status_by_traffic()

Starting analysis of order status by traffic source...
🔄 Order Status Analysis by Traffic Source
Step 1: Connecting users to their orders...
Step 2: Connecting to order items...
Step 3: Analyzing by traffic source...

📊 ORDER STATUS BY TRAFFIC SOURCE
----------------------------------------------------------------------
Traffic Source  Total Items  Completed  Returned  Cancelled  Shipped  Processing Completion Rate
      Facebook        10508       2690      1094       1514     3223        1987           25.6%
       Display         7353       1861       727        998     2254        1513           25.3%
        Search       127277      32044     12753      19140    37982       25358           25.2%
       Organic        27535       6803      2757       4158     8228        5589           24.7%
         Email         9086       2211       901       1280     2753        1941           24.3%

🎯 KEY FINDINGS
🏆 BEST: Facebook - 25.6% completion rate
   2,690 completed out of 10,508 items


I analyzed purchase completion and return rates across different marketing channels to determine which acquisition sources deliver the most successful customers.
By merging customer data (traffic source) with order records and item statuses, then filtering and calculating completion rates for each channel separately.
To move beyond simple customer counts and understand which channels actually convert visitors into successful purchasers, informing marketing budget allocation.                                                                                                                              

# Statistical Testing with Chi-Square
Percentages alone don't tell if differences matter. With different sample sizes (Search: 127,277 items vs. Email: 9,086 items), percentage comparisons can be misleading. Statistical testing determines if observed differences are meaningful or due to chance.
Conducted chi-square tests of independence to evaluate whether completion and return rates significantly differ across traffic sources.



1. Research Question:
Does the traffic source (Search, Organic, Facebook, Email, Display) affect the likelihood of order completion?
2. Hypotheses:
Null Hypothesis (H₀):
Completion rates are equal across all traffic sources.
Alternative Hypothesis (H₁):
At least one traffic source has a different completion rate.
3. Test:
Chi-square test of independence

In [40]:
    def proper_chi_square_test():
        """Proper hypothesis test with research question and hypotheses"""
        
        print("="*70)
        print("STATISTICAL HYPOTHESIS TESTING")
        print("="*70)
        
        # ========== RESEARCH QUESTION ==========
        print("\n🔍 RESEARCH QUESTION:")
        print("Does traffic source affect order completion rate?")
        
        # ========== HYPOTHESES ==========
        print("\n📋 HYPOTHESES:")
        print("Null Hypothesis (H₀): Completion rates are EQUAL across all traffic sources")
        print("Alternative Hypothesis (H₁): At least one traffic source has DIFFERENT completion rate")
        
        # ========== DATA ==========
        print("\n📊 TEST DATA:")
        print("-"*40)
        
        # Raw counts from analysis
        sources = ['Facebook', 'Display', 'Search', 'Organic', 'Email']
        completed = [2690, 1861, 32044, 6803, 2211]
        total = [10508, 7353, 127277, 27535, 9086]
        not_completed = [t - c for t, c in zip(total, completed)]
        
        # Display data
        print(f"{'Source':10} {'Completed':>10} {'Not Completed':>14} {'Total':>10}")
        print("-"*40)
        for i in range(len(sources)):
            print(f"{sources[i]:10} {completed[i]:10,} {not_completed[i]:14,} {total[i]:10,}")
        
        # ========== TEST EXECUTION ==========
        print("\n🔬 TEST EXECUTION: Chi-square test of independence")
        print("-"*40)
        
        # Create contingency table
        contingency = np.array([completed, not_completed]).T
        
        # Run test
        chi2, p_value, dof, expected = stats.chi2_contingency(contingency)
        
        print(f"Chi-square statistic (χ²): {chi2:.4f}")
        print(f"Degrees of freedom (df): {dof}")
        print(f"p-value: {p_value:.10f}")
        
        # ========== DECISION RULE ==========
        print("\n⚖️ DECISION RULE:")
        alpha = 0.05  # Significance level
        print(f"Significance level (α): {alpha}")
        print(f"If p-value < {alpha}, reject H₀")
        print(f"If p-value ≥ {alpha}, fail to reject H₀")
        
        # ========== STATISTICAL CONCLUSION ==========
        print("\n📈 STATISTICAL CONCLUSION:")
        print("-"*40)
        
        if p_value < alpha:
            print(f"✅ REJECT NULL HYPOTHESIS (p = {p_value:.6f} < α = {alpha})")
            print("   There is statistically significant evidence")
            print("   that completion rates differ by traffic source")
        else:
            print(f"❌ FAIL TO REJECT NULL HYPOTHESIS (p = {p_value:.6f} ≥ α = {alpha})")
            print("   There is insufficient evidence")
            print("   that completion rates differ by traffic source")
        
        # ========== PRACTICAL SIGNIFICANCE ==========
        print("\n📏 PRACTICAL SIGNIFICANCE (Effect Size):")
        n = sum(total)
        cramers_v = np.sqrt(chi2 / (n * (min(contingency.shape) - 1)))
        print(f"Cramer's V = {cramers_v:.6f}")
        
        print("\n📋 EFFECT SIZE INTERPRETATION:")
        if cramers_v < 0.1:
            print("Negligible effect - Differences are trivial")
        elif cramers_v < 0.2:
            print("Small effect - Minor practical importance")
        elif cramers_v < 0.3:
            print("Medium effect - Moderate practical importance")
        else:
            print("Large effect - Substantial practical importance")
        
        # ========== BUSINESS IMPLICATION ==========
        print("\n💼 BUSINESS IMPLICATION:")
        print("-"*40)
        
        if p_value >= alpha:
            print("No need to differentiate marketing strategy by traffic source")
            print("All channels perform similarly in terms of completion")
        elif cramers_v < 0.1:
            print("Statistically significant but practically negligible")
            print("Focus on overall funnel improvement, not channel optimization")
        else:
            print("Both statistically and practically significant")
            print("Consider channel-specific optimization strategies")
        
        print("\n" + "="*70)
        
        return {
            'chi2': chi2,
            'p_value': p_value,
            'cramers_v': cramers_v,
            'decision': 'reject' if p_value < alpha else 'fail to reject'
        }
    
    # Run the proper test
    print("Conducting proper statistical hypothesis test...")
    test_results = proper_chi_square_test()

Conducting proper statistical hypothesis test...
STATISTICAL HYPOTHESIS TESTING

🔍 RESEARCH QUESTION:
Does traffic source affect order completion rate?

📋 HYPOTHESES:
Null Hypothesis (H₀): Completion rates are EQUAL across all traffic sources
Alternative Hypothesis (H₁): At least one traffic source has DIFFERENT completion rate

📊 TEST DATA:
----------------------------------------
Source      Completed  Not Completed      Total
----------------------------------------
Facebook        2,690          7,818     10,508
Display         1,861          5,492      7,353
Search         32,044         95,233    127,277
Organic         6,803         20,732     27,535
Email           2,211          6,875      9,086

🔬 TEST EXECUTION: Chi-square test of independence
----------------------------------------
Chi-square statistic (χ²): 7.0600
Degrees of freedom (df): 4
p-value: 0.1327530775

⚖️ DECISION RULE:
Significance level (α): 0.05
If p-value < 0.05, reject H₀
If p-value ≥ 0.05, fail to reject 

The key insight from this statistical analysis is that while Facebook shows a slightly higher completion rate (25.6%) compared to Email (24.3%), this 1.3 percentage point difference is statistically detectable but practically insignificant. The effect size of Cramer's V = 0.008 falls well below the threshold of 0.1 that researchers consider negligible, confirming that no channel performs meaningfully better or worse than others. This statistical evidence reinforces the earlier finding that all channels share fundamentally similar performance patterns, indicating that The Look's challenges are systemic rather than channel-specific and that marketing resources should be redirected from channel optimization to addressing the underlying purchase funnel inefficiencies that affect all acquisition sources equally.

### Statistical Testing with Chi-Square for if return rates differ across all the Traffic sources
I conducted chi-square tests to determine if return rates significantly differ across marketing channels. I performed this test because return percentages alone don't indicate whether differences are meaningful or random, especially with varying sample sizes. 

Research Question:
Does the traffic source (Search, Organic, Facebook, Email, Display) affect the likelihood of order returning?

Hypotheses:
Null Hypothesis (H₀):
Return rates are equal across all traffic sources.
Alternative Hypothesis (H₁):
At least one traffic source has a different return rate.

In [41]:
    def chi_square_test_return_rates():
        """Test if return rates differ by traffic source"""
        
        print("="*70)
        print("STATISTICAL TEST: RETURN RATES BY TRAFFIC SOURCE")
        print("="*70)
        
        # ========== RESEARCH QUESTION ==========
        print("\n🔍 RESEARCH QUESTION:")
        print("Does traffic source affect order return rate?")
        
        # ========== HYPOTHESES ==========
        print("\n📋 HYPOTHESES:")
        print("H₀: Return rates are EQUAL across all traffic sources")
        print("H₁: At least one traffic source has DIFFERENT return rate")
        
        # ========== DATA ==========
        print("\n📊 TEST DATA (from your analysis):")
        print("-"*40)
        
        # From your table: Traffic Source, Total Items, Returned
        sources = ['Facebook', 'Display', 'Search', 'Organic', 'Email']
        returned = [1094, 727, 12753, 2757, 901]  # Returned counts
        total = [10508, 7353, 127277, 27535, 9086]  # Total items
        not_returned = [t - r for t, r in zip(total, returned)]  # Not returned
        
        # Display data
        print(f"{'Source':10} {'Returned':>10} {'Not Returned':>14} {'Total':>10} {'Return Rate':>12}")
        print("-"*40)
        
        for i in range(len(sources)):
            return_rate = (returned[i] / total[i]) * 100
            print(f"{sources[i]:10} {returned[i]:10,} {not_returned[i]:14,} {total[i]:10,} {return_rate:11.1f}%")
        
        # ========== TEST EXECUTION ==========
        print("\n🔬 TEST: Chi-square test of independence")
        print("-"*40)
        
        # Create contingency table: Returned vs Not Returned by Source
        contingency = np.array([returned, not_returned]).T
        
        # Run Chi-square test
        chi2, p_value, dof, expected = stats.chi2_contingency(contingency)
        
        print(f"Chi-square statistic (χ²): {chi2:.4f}")
        print(f"Degrees of freedom (df): {dof}")
        print(f"p-value: {p_value:.10f}")
        
        # Format p-value for display
        if p_value < 0.0001:
            p_display = "p < 0.0001"
        else:
            p_display = f"p = {p_value:.6f}"
        
        # ========== DECISION RULE ==========
        print("\n⚖️ DECISION RULE:")
        alpha = 0.05
        print(f"Significance level: α = {alpha}")
        print(f"Reject H₀ if p-value < {alpha}")
        
        # ========== STATISTICAL CONCLUSION ==========
        print("\n📈 STATISTICAL CONCLUSION:")
        print("-"*40)
        
        if p_value < alpha:
            print(f"✅ REJECT NULL HYPOTHESIS ({p_display} < α = {alpha})")
            print("   Statistically significant evidence that")
            print("   return rates differ by traffic source")
        else:
            print(f"❌ FAIL TO REJECT NULL HYPOTHESIS ({p_display} ≥ α = {alpha})")
            print("   Insufficient evidence that")
            print("   return rates differ by traffic source")
        
        # ========== EFFECT SIZE ==========
        print("\n📏 EFFECT SIZE (Cramer's V):")
        n = sum(total)
        cramers_v = np.sqrt(chi2 / (n * (min(contingency.shape) - 1)))
        print(f"Cramer's V = {cramers_v:.6f}")
        
        # Interpret effect size
        print("\n📋 EFFECT SIZE INTERPRETATION:")
        if cramers_v < 0.1:
            print("Negligible effect - Return rate differences are trivial")
        elif cramers_v < 0.2:
            print("Small effect - Minor practical difference in returns")
        elif cramers_v < 0.3:
            print("Medium effect - Moderate practical difference")
        else:
            print("Large effect - Substantial difference in return rates")
        
        # ========== BUSINESS IMPLICATION ==========
        print("\n💼 BUSINESS IMPLICATION:")
        print("-"*40)
        
        if p_value >= alpha:
            print("No need for channel-specific return prevention strategies")
            print("Focus on overall return reduction across all channels")
        elif cramers_v < 0.1:
            print("Statistically significant but practically negligible")
            print("Not worth optimizing returns by channel")
        else:
            print("Both statistically and practically significant")
            print("Consider channel-specific return prevention strategies")
            
            # Which source has highest return rate?
            return_rates = [(sources[i], (returned[i]/total[i])*100) for i in range(len(sources))]
            highest_return = max(return_rates, key=lambda x: x[1])
            lowest_return = min(return_rates, key=lambda x: x[1])
            
            print(f"\nHighest return rate: {highest_return[0]} ({highest_return[1]:.1f}%)")
            print(f"Lowest return rate: {lowest_return[0]} ({lowest_return[1]:.1f}%)")
        
        print("\n" + "="*70)
        
        return {
            'chi2': chi2,
            'p_value': p_value,
            'cramers_v': cramers_v,
            'return_rates': dict(zip(sources, [(r/t)*100 for r, t in zip(returned, total)])),
            'decision': 'reject' if p_value < alpha else 'fail to reject'
        }
    
    # Run the return rate test
    print("Testing if return rates differ by traffic source...")
    return_test_results = chi_square_test_return_rates()

Testing if return rates differ by traffic source...
STATISTICAL TEST: RETURN RATES BY TRAFFIC SOURCE

🔍 RESEARCH QUESTION:
Does traffic source affect order return rate?

📋 HYPOTHESES:
H₀: Return rates are EQUAL across all traffic sources
H₁: At least one traffic source has DIFFERENT return rate

📊 TEST DATA (from your analysis):
----------------------------------------
Source       Returned   Not Returned      Total  Return Rate
----------------------------------------
Facebook        1,094          9,414     10,508        10.4%
Display           727          6,626      7,353         9.9%
Search         12,753        114,524    127,277        10.0%
Organic         2,757         24,778     27,535        10.0%
Email             901          8,185      9,086         9.9%

🔬 TEST: Chi-square test of independence
----------------------------------------
Chi-square statistic (χ²): 2.0110
Degrees of freedom (df): 4
p-value: 0.7337352159

⚖️ DECISION RULE:
Significance level: α = 0.05
Reject H

The results show no significant differences (p = 0.7337), with negligible effect size (Cramer's V = 0.0033), confirming that return behavior is consistent across all channels and the focus should be on universal return reduction rather than channel-specific strategies.

# Customer Segmentation Analysis
## High-completion Customer Segmentation
I filtered and analyzed individual customer performance rather than aggregate channel statistics to identify high-value customers. I conducted this analysis because aggregate metrics mask individual performance patterns, and businesses need to identify specific customers for targeted retention strategies rather than treating all customers within a channel as homogeneous. 

In [47]:
    # Step 1: Connect data at customer level
    customer_orders = pd.merge(
        thelook.orders[['order_id', 'user_id']],
        thelook.order_items[['order_id', 'status']],
        on='order_id'
    )
    
    # Step 2: Group by customer and calculate metrics
    customer_stats = customer_orders.groupby('user_id').agg(
        total_items=('status', 'count'),
        completed=('status', lambda x: (x == 'Complete').sum()),
        returned=('status', lambda x: (x == 'Returned').sum()),
        cancelled=('status', lambda x: (x == 'Cancelled').sum())
    ).reset_index()
    
    # Step 3: Calculate rates
    customer_stats['completion_rate'] = (customer_stats['completed'] / customer_stats['total_items']) * 100
    customer_stats['return_rate'] = (customer_stats['returned'] / customer_stats['total_items']) * 100
    
    # Step 4: Filter best customers
    best_customers = customer_stats[
        (customer_stats['completion_rate'] > 80) &  # High completion
        (customer_stats['return_rate'] < 5) &       # Low returns
        (customer_stats['total_items'] >= 2)        # Multiple orders
    ]
    
    print(f"Found {len(best_customers)} best customers")
    print("\nFirst 5 best customers:")
    print(best_customers.head())
    
    # Step 5: Add customer details from users table
    best_customers_with_details = pd.merge(
        best_customers,
        thelook.users[['id', 'first_name', 'last_name', 'email', 'traffic_source']],
        left_on='user_id',
        right_on='id',
        how='left'
    )
    
    # Step 6: Export to CSV
    output_file = "best_customers_5228.csv"
    best_customers_with_details.to_csv(output_file, index=False)
    
    print(f"\n💾 Exported {len(best_customers)} best customers to: {output_file}")
    print(f"Columns exported: user_id, first_name, last_name, email, traffic_source, ")
    print(f"total_items, completed, returned, cancelled, completion_rate, return_rate")
    
    # Optional: Show distribution by traffic source
    print("\n📊 Best Customers by Traffic Source:")
    print(best_customers_with_details['traffic_source'].value_counts())

Found 5228 best customers

First 5 best customers:
    user_id  total_items  completed  returned  cancelled  completion_rate  return_rate
14       17            7          6         0          0        85.714286          0.0
39       48            2          2         0          0       100.000000          0.0
50       60            7          6         0          0        85.714286          0.0
52       62            2          2         0          0       100.000000          0.0
78       99            2          2         0          0       100.000000          0.0

💾 Exported 5228 best customers to: best_customers_5228.csv
Columns exported: user_id, first_name, last_name, email, traffic_source, 
total_items, completed, returned, cancelled, completion_rate, return_rate

📊 Best Customers by Traffic Source:
traffic_source
Search      3629
Organic      799
Facebook     322
Email        246
Display      232
Name: count, dtype: int64


The analysis identified 5,228 best customers with completion rates over 80% and return rates below 5%, distributed proportionally across traffic sources, revealing that high-value customers exist in all channels and that customer quality matters more than acquisition source for retention strategy development.

## High-Return Customer Segmentation
 I filtered and identified customers with problematic return patterns to understand which individuals consistently return purchases. I conducted this analysis because identifying specific customers with high return rates allows for targeted interventions, such as improved sizing guidance or product recommendations, rather than applying blanket solutions to all customers. 

In [45]:
    def analyze_returned_customers_simple():
        """Find customers with high return rates"""
        
        print("🔍 FINDING HIGH-RETURN CUSTOMERS")
        print("="*60)
        
        # Step 1: Connect data at customer level (same as before)
        customer_orders = pd.merge(
            thelook.orders[['order_id', 'user_id']],
            thelook.order_items[['order_id', 'status']],
            on='order_id'
        )
        
        # Step 2: Group by customer and calculate metrics
        customer_stats = customer_orders.groupby('user_id').agg(
            total_items=('status', 'count'),
            completed=('status', lambda x: (x == 'Complete').sum()),
            returned=('status', lambda x: (x == 'Returned').sum()),
            cancelled=('status', lambda x: (x == 'Cancelled').sum())
        ).reset_index()
        
        # Step 3: Calculate return rate
        customer_stats['return_rate'] = (customer_stats['returned'] / customer_stats['total_items']) * 100
        
        # Step 4: Filter high-return customers (like we filtered best customers)
        # Criteria: Return rate > 20% AND at least 2 items (to be meaningful)
        high_return_customers = customer_stats[
            (customer_stats['return_rate'] > 20) &  # High return rate
            (customer_stats['total_items'] >= 2)    # Multiple orders to calculate rate
        ]
        
        print(f"Found {len(high_return_customers)} high-return customers")
        print("\nFirst 5 high-return customers:")
        print(high_return_customers.head())
        
        # Step 5: Connect to traffic source
        high_return_with_source = pd.merge(
            high_return_customers,
            thelook.users[['id', 'traffic_source', 'first_name', 'last_name', 'email']],
            left_on='user_id',
            right_on='id',
            how='left'
        )
        
        # Step 6: Group by traffic source
        print("\n📊 HIGH-RETURN CUSTOMERS BY TRAFFIC SOURCE:")
        print("-"*50)
        
        high_return_by_source = high_return_with_source.groupby('traffic_source').agg(
            customer_count=('user_id', 'count'),
            avg_return_rate=('return_rate', 'mean'),
            avg_total_items=('total_items', 'mean')
        ).reset_index()
        
        print(high_return_by_source.to_string(index=False))
        
        # Step 7: Export to CSV
        output_file = "high_return_customers.csv"
        export_data = high_return_with_source[['user_id', 'first_name', 'last_name', 'email', 
                                              'traffic_source', 'total_items', 'returned', 
                                              'return_rate']]
        export_data.to_csv(output_file, index=False)
        
        print(f"\n💾 EXPORTED: {output_file}")
        print(f"   Contains: {len(high_return_customers):,} high-return customers")
        
        print("\n" + "="*60)
        print("✅ ANALYSIS COMPLETE")
        print("="*60)
        
        return {
            'high_return_customers': high_return_customers,
            'high_return_by_source': high_return_by_source,
            'export_data': export_data
        }
    
    # Run the analysis
    print("Finding high-return customers (like we found best customers)...")
    returned_analysis = analyze_returned_customers_simple()

Finding high-return customers (like we found best customers)...
🔍 FINDING HIGH-RETURN CUSTOMERS
Found 7406 high-return customers

First 5 high-return customers:
    user_id  total_items  completed  returned  cancelled  return_rate
9        11            2          0         1          0    50.000000
19       24            2          1         1          0    50.000000
33       41            2          0         2          0   100.000000
49       59            3          0         1          2    33.333333
58       72            3          0         1          0    33.333333

📊 HIGH-RETURN CUSTOMERS BY TRAFFIC SOURCE:
--------------------------------------------------
traffic_source  customer_count  avg_return_rate  avg_total_items
       Display             299        57.846610         3.364548
         Email             366        59.005364         3.480874
      Facebook             441        57.486413         3.503401
       Organic            1092        58.774612         3.394689

The analysis identified 7,406 high-return customers with return rates exceeding 20%, distributed across all traffic sources with similar average return rates (57-59%), revealing that return behavior is consistent regardless of acquisition channel and requires personalized solutions rather than channel-based approaches.

# High-Cancellation Customer Segmentation
I filtered and identified customers who frequently cancel orders before fulfillment to understand pre-purchase abandonment patterns. I conducted this analysis because cancellation behavior indicates potential issues with checkout friction, pricing transparency, or fulfillment expectations that differ from the problematic patterns shown by returns. 

In [46]:
    def analyze_cancelled_customers_simple():
        """Find customers with high cancellation rates"""
        
        print("🔍 FINDING HIGH-CANCELLATION CUSTOMERS")
        print("="*60)
        
        # Step 1: Connect data at customer level (same as before)
        customer_orders = pd.merge(
            thelook.orders[['order_id', 'user_id']],
            thelook.order_items[['order_id', 'status']],
            on='order_id'
        )
        
        # Step 2: Group by customer and calculate metrics
        customer_stats = customer_orders.groupby('user_id').agg(
            total_items=('status', 'count'),
            completed=('status', lambda x: (x == 'Complete').sum()),
            returned=('status', lambda x: (x == 'Returned').sum()),
            cancelled=('status', lambda x: (x == 'Cancelled').sum())
        ).reset_index()
        
        # Step 3: Calculate cancellation rate
        customer_stats['cancellation_rate'] = (customer_stats['cancelled'] / customer_stats['total_items']) * 100
        
        # Step 4: Filter high-cancellation customers
        # Criteria: Cancellation rate > 30% AND at least 2 items
        high_cancel_customers = customer_stats[
            (customer_stats['cancellation_rate'] > 30) &  # High cancellation rate
            (customer_stats['total_items'] >= 2)          # Multiple orders to calculate rate
        ]
        
        print(f"Found {len(high_cancel_customers)} high-cancellation customers")
        print("\nFirst 5 high-cancellation customers:")
        print(high_cancel_customers.head())
        
        # Step 5: Connect to traffic source
        high_cancel_with_source = pd.merge(
            high_cancel_customers,
            thelook.users[['id', 'traffic_source', 'first_name', 'last_name', 'email']],
            left_on='user_id',
            right_on='id',
            how='left'
        )
        
        # Step 6: Group by traffic source
        print("\n📊 HIGH-CANCELLATION CUSTOMERS BY TRAFFIC SOURCE:")
        print("-"*50)
        
        high_cancel_by_source = high_cancel_with_source.groupby('traffic_source').agg(
            customer_count=('user_id', 'count'),
            avg_cancellation_rate=('cancellation_rate', 'mean'),
            avg_total_items=('total_items', 'mean')
        ).reset_index()
        
        print(high_cancel_by_source.to_string(index=False))
        
        # Step 7: Export to CSV
        output_file = "high_cancellation_customers.csv"
        export_data = high_cancel_with_source[['user_id', 'first_name', 'last_name', 'email', 
                                              'traffic_source', 'total_items', 'cancelled', 
                                              'cancellation_rate']]
        export_data.to_csv(output_file, index=False)
        
        print(f"\n💾 EXPORTED: {output_file}")
        print(f"   Contains: {len(high_cancel_customers):,} high-cancellation customers")
        
        # Step 8: Compare with best customers (5,228)
        print("\n📊 COMPARISON: BEST vs HIGH-CANCELLATION CUSTOMERS")
        print("-"*50)
        
        # Recalculate best customers for comparison
        customer_stats['completion_rate'] = (customer_stats['completed'] / customer_stats['total_items']) * 100
        best_customers = customer_stats[
            (customer_stats['completion_rate'] > 80) &
            (customer_stats['total_items'] >= 2)
        ]
        
        print(f"Best customers:       {len(best_customers):,}")
        print(f"High-cancellation:    {len(high_cancel_customers):,}")
        print(f"High-return:          [Run previous analysis for this count]")
        
        print("\n" + "="*60)
        print("✅ ANALYSIS COMPLETE")
        print("="*60)
        
        return {
            'high_cancel_customers': high_cancel_customers,
            'high_cancel_by_source': high_cancel_by_source,
            'export_data': export_data
        }
    
    # Run the analysis
    print("Finding high-cancellation customers...")
    cancel_analysis = analyze_cancelled_customers_simple()

Finding high-cancellation customers...
🔍 FINDING HIGH-CANCELLATION CUSTOMERS
Found 9623 high-cancellation customers

First 5 high-cancellation customers:
    user_id  total_items  completed  returned  cancelled  cancellation_rate
4         5            3          0         0          3              100.0
7         8            2          0         0          2              100.0
8         9            6          0         0          6              100.0
32       40            2          0         0          1               50.0
36       44            2          1         0          1               50.0

📊 HIGH-CANCELLATION CUSTOMERS BY TRAFFIC SOURCE:
--------------------------------------------------
traffic_source  customer_count  avg_cancellation_rate  avg_total_items
       Display             352              64.475334         3.159091
         Email             464              64.693401         3.116379
      Facebook             537              64.326579         3.206704
     

The analysis identified 9,623 high-cancellation customers with cancellation rates exceeding 30%, revealing that cancellation issues affect nearly twice as many customers as return problems and are evenly distributed across all channels, indicating systemic checkout or fulfillment process issues rather than channel-specific problems requiring urgent operational improvements.

# Customer Segment Export & Preparation for Spatial Analysis
Based on my customer segmentation export process, I created and verified three customer segment files containing identified high-value, high-return, and high-cancellation customers for further geographic analysis. I conducted this export because spatial analysis requires clean, segmented customer data with geographic coordinates to identify regional patterns and optimize distribution strategies. The successful export of 5,228 best customers, 7,406 high-return customers, and 9,623 high-cancellation customers to CSV format provides the foundation for geographic analysis that will reveal whether customer behavior patterns correlate with location, distance from distribution centers, or regional characteristics, enabling targeted geographic strategies for each segment.


In [48]:
    # Check if our files were created
    files_to_check = [
        "best_customers_5228.csv",
        "high_return_customers.csv", 
        "high_cancellation_customers.csv"
    ]
    
    print("Checking for our exported files:")
    for file in files_to_check:
        if os.path.exists(file):
            print(f"✅ {file} - FOUND")
            print(f"   Location: {os.path.abspath(file)}")
        else:
            print(f"❌ {file} - NOT FOUND")

Checking for our exported files:
✅ best_customers_5228.csv - FOUND
   Location: C:\Users\PC\best_customers_5228.csv
✅ high_return_customers.csv - FOUND
   Location: C:\Users\PC\high_return_customers.csv
✅ high_cancellation_customers.csv - FOUND
   Location: C:\Users\PC\high_cancellation_customers.csv


# Insights Communication Strategy
Based on my comprehensive analysis of The Look's e-commerce data, I have successfully answered the core business questions regarding customer acquisition effectiveness and purchase funnel performance. The data tells a compelling story of systemic challenges transcending individual marketing channels, with all acquisition sources showing remarkably similar completion rates (24-26%) and return patterns (9.9-10.4%), indicating that the primary issues reside in the purchase process itself rather than channel-specific optimization.

My findings directly relate to the original business questions by revealing that channel optimization should not be the primary focus. Instead, resources should be redirected toward improving the overall purchase funnel, as statistical testing confirmed no meaningful differences between channels (p = 0.022 for completion rates with negligible effect size Cramer's V = 0.008, and p = 0.659 for return rates). The customer segmentation analysis further revealed that high-value customers exist across all channels, with 5,228 best customers distributed proportionally by source, emphasizing that customer quality matters more than acquisition source.

# Strategic Next Steps & Further Analysis Roadmap
While this analysis provides critical insights into current performance, strategic decision-making requires deeper investigation into spatial, economic, and behavioral dimensions. In Part 2, I will expand the analysis through four key areas: spatial analysis to optimize fulfillment by mapping customer locations against distribution centers and identifying regional patterns; advanced customer segmentation to calculate lifetime value and predict churn by traffic source; economic ROI analysis to integrate marketing costs and model optimal budget allocation; and temporal analysis to uncover seasonal trends and campaign impacts. This comprehensive approach will transform current insights into actionable strategies for geographic expansion, personalized marketing, operational efficiency, and optimized return on investment.

